In [ ]:
# 导入包 

import sys
import pathlib
sys.path.append(str(pathlib.Path(__file__).parent.parent.parent))

import math
import matplotlib.pyplot as plt
import numpy as np

from utils.angle import rot_mat_2d


In [ ]:
#  Simulation parameter
DT=0.1 # time step
SIM_TIME = 50.0# 模拟时长


GPS_NOISE = np.diag([0.5,0.5])**2
INPUT_NOISE = np.diag([1.0,np.deg2rad(30.0)])**2

show_animation =True
Q = np.diag([
    0.1,  # variance of location on x-axis
    0.1,  # variance of location on y-axis
    np.deg2rad(1.0),  # variance of yaw angle
    1.0  # variance of velocity
]) ** 2  # predict state covariance
R = np.diag([1.0, 1.0]) ** 2  # Observation x,y position covariance


In [ ]:
def main():
    print(__file__ + " start!!")

    time = 0.0

    ## 状态向量
    xEst = np.zeros((4,1))
    xTrue = np.zeros((4,1))
    PEst = np.eye(4)# 协方差矩阵

    xDR = np.zeros((4,1))

    # 旧值 history
    hxEst = xEst
    hxTrue = xTrue
    hxDR = xTrue
    hz = np.zeros((2,1))

    while SIM_TIME>=time:
        time +=DT
        u =calc_input()
        
        xTrue,z,xDR,ud = observation(xTrue,xDR,u)

        xEst,PEst = ekf_estimation(xEst,PEst,z,ud)
    

In [ ]:
def ekf_estimation(xEst,PEst,z,ud):
    xPred = motion_model(xEst,u)
    jF = jacob_f(xEst,u)
    Ppred = jF @ PEst@ jF.T +Q

    jH = jacob_h()

In [2]:
def observation(xTrue,xd,u):
    xTrue = motion_model(xTrue,u)

    z = observation_model(xTrue)+GPS_NOISE @np.random.randn(2,1)

    ud = u+INPUT_NOISE @np.random.randn(2,1)

    xd = motion_model(xd,ud)

    return xTrue,z,xd,ud

In [3]:
def motion_model(x,u):
    F = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,0]])
    
    B = np.array([math.cos(x[2,0])*DT,0]
                 [math.sin(x[2,0])*DT,0],
                 [0,DT],
                 [1,0])
    
    x = F@x +B@u #用于计算下一步骤的状态，
    #执行矩阵乘法运算
    return x

<>:7: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:7: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\Yao\AppData\Local\Temp\ipykernel_3892\232248018.py:7: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  B = np.array([math.cos(x[2,0])*DT,0]


In [4]:
def observation_model(x):
    H = np.array([1,0,0,0],
                 [0,1,0,0])
    
    z = H@x
    return z

In [5]:
def jacob_f(x,u):
    v = u[0,0]

    jF = np.array(
        [1.0,0.0,-DT*v*math.sin(x[2,0]),DT*math.cos(x[2,0])],
        [0.0,1.0,v*math.cos(x[2,0])*DT,DT*math.sin(x[2,0])],
        [0.0,1.0,1.0,0.0],
        [0.0,0.0,0.0,1.0]
    )

    return jF

In [6]:
def calc_input():
    v = 1.0
    yawrate = 0.1
    u = np.array([[v],[yawrate]])

    return u